In [ ]:
import pandas as pd
import numpy as np
import sys
import os
sys.path.append(os.path.abspath(".."))
from core.viz import plot_line, create_subplot_grid, plot_bar
from core.s3 import S3AssetManager

In [17]:
notebook_name = "adipell_honduras"
s3 = S3AssetManager(notebook_name=notebook_name)

In [18]:
base_path = "raw/honduras"
df_honduras = s3.read_excel(f"{base_path}/Pruebas Adipell CMI Honduras 250917 (1).xlsx", sheet_name='orden')
df_honduras

Reading Excel from: s3://galileo-c4e9a2f1/raw/honduras/Pruebas Adipell CMI Honduras 250917 (1).xlsx


,ref,producto,x,punto,peso muestra,PDI,finos_g,finos_pct,malla _18_g,malla _18_pct,malla_10_g,malla_10_pct,malla_6_g,malla_6_pct,t/h,A_pct,A_g,P,T
0,1,con adipell,Vitaengorde fin,dado,741.0,NaN,16.4,0.022132,724.6,0.977868,NaN,NaN,NaN,NaN,26.15,0.810,699.0,2.3,59.0
1,1,con adipell,Vitaengorde fin,enfriador,1131.7,93.60,4.5,0.003976,10.8,0.009543,18.4,0.016259,1093.4,0.966157,NaN,NaN,NaN,NaN,NaN
2,1,con adipell,Vitaengorde fin,empacado,1245.5,96.66,25.9,0.020795,32.7,0.026255,25.8,0.020715,1156.4,0.928462,NaN,NaN,NaN,NaN,NaN
3,2,con adipell,Vitaengorde fin,dado,930.2,NaN,18.5,0.019888,911.7,0.980112,NaN,NaN,NaN,NaN,27.26,0.805,701.0,2.3,56.0
4,2,con adipell,Vitaengorde fin,enfriador,1939.5,92.98,10.4,0.005362,25.1,0.012941,31.8,0.016396,1867.5,0.962877,NaN,NaN,NaN,NaN,NaN
5,2,con adipell,Vitaengorde fin,empacado,1775.6,97.10,20.2,0.011376,18.1,0.010194,15.8,0.008898,1716.8,0.966884,NaN,NaN,NaN,NaN,NaN
6,3,con adipell,Vitaengorde fin,dado,939.8,NaN,14.7,0.015642,925.1,0.984358,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,3,con adipell,Vitaengorde fin,enfriador,1662.1,92.68,6.2,0.003730,18.0,0.010830,15.7,0.009446,1607.1,0.966909,NaN,NaN,NaN,NaN,NaN
8,3,con adipell,Vitaengorde fin,empacado,1648.2,96.74,26.6,0.016139,37.3,0.022631,30.8,0.018687,1547.9,0.939146,NaN,NaN,NaN,NaN,NaN
9,2 (2),con adipell,Vitaengorde fin,dado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
df_finos_punto = df_honduras[df_honduras["punto"] == "enfriador"].groupby(["producto"]).agg(

    finos_pct=('finos_pct', 'mean'),
).reset_index()
df_finos_punto["finos_pct"] = df_finos_punto["finos_pct"]*100
df_finos_punto

,producto,finos_pct
0,con adipell,0.435625
1,sin adipell,0.454234


In [20]:
df_honduras_group = df_honduras.groupby(["producto"]).agg(
    pdi=('PDI', 'mean'),
    #finos_pct=('finos_pct', 'mean'),
    rendimiento=('t/h', 'mean'),
    A_pct=('A_pct', 'mean'),
    presion=('P', 'mean'),
    temperatura=("T", "mean"),
).reset_index()


df_honduras_group = pd.merge(df_honduras_group, df_finos_punto, on=["producto"])
df_honduras_group["experimento"] = "Exp I"
df_honduras_group

,producto,pdi,rendimiento,A_pct,presion,temperatura,finos_pct,experimento
0,con adipell,95.240000,26.705,0.807500,2.3,57.500000,0.435625,Exp I
1,sin adipell,95.122857,25.585,0.789333,2.3,61.333333,0.454234,Exp I


In [37]:
fig_pdi =plot_bar(
df=df_honduras_group,
x_col="experimento",
y_col="pdi",
group_col="producto",
delta_reference_group='sin adipell',
show_delta=True,
delta_type='delta',
#title="PDI (%)",
)
fig_pdi.show()
s3.save_plotly_html(fig_pdi, "pdi_honduras.html")

In [38]:
fig_rendimiento =plot_bar(
df=df_honduras_group,
x_col="experimento",
y_col="rendimiento",
group_col="producto",
delta_reference_group='sin adipell',
show_delta=True,
delta_type='pct',
#title="Rendimiento (Ton/H)",
)
fig_rendimiento.show()
s3.save_plotly_html(fig_rendimiento, "rendimiento_honduras.html")

In [39]:
fig_final = create_subplot_grid(
    figures=[fig_rendimiento, fig_pdi],
    rows=1, cols=2,
    titles=["Rendimiento (Ton/H)", "PDI (%)"],
    main_title="Métricas de rendimiento y calidad",
    shared_y=False,
    width=1000, height=400
)

fig_final.show()
s3.save_plotly_html(fig_final, "pdi_finos_rendimiento_honduras.html")

In [40]:
df_finos_punto = df_honduras.groupby(["producto", "punto"]).agg(
    pdi=('PDI', 'mean'),
    finos_pct=('finos_pct', 'mean'),
    malla_18_pct=('malla _18_pct', 'mean'),
    malla_10_pct=('malla_10_pct', "mean"),
    malla_6_pct=('malla_6_pct', 'mean'),
).reset_index()
df_finos_punto

,producto,punto,pdi,finos_pct,malla_18_pct,malla_10_pct,malla_6_pct
0,con adipell,dado,NaN,0.019221,0.980779,NaN,NaN
1,con adipell,empacado,96.855000,0.020043,0.022754,0.016567,0.937087
2,con adipell,enfriador,93.086667,0.004356,0.011105,0.014034,0.965314
3,sin adipell,dado,NaN,0.018329,0.981970,NaN,NaN
4,sin adipell,empacado,96.245000,0.008909,0.009593,0.010191,0.964392
5,sin adipell,enfriador,93.626667,0.004542,0.010918,0.011380,0.965115


In [41]:
(26.7-25.6)

1.0999999999999979

In [42]:
cols_config = {
    "con adipell": "#1C8074", 
    "sin adipell": "#666666"
}

puntos_order = ["dado", "enfriador", "empacado"]  # o el orden que quieras

fig1 = plot_line(
    df=df_finos_punto,
    x_col="punto",                # Antes estaba implícito ("punto")
    y_col="finos_pct",            # Tu metric_col
   group_col="producto",     # La columna que separa las líneas ("con" vs "sin")
   line_colors=cols_config,    # Aquí inyectas los colores corporativos
    order_x=puntos_order,         # El orden de los pasos
    title="Finos (%) por punto de proceso — Con vs Sin Adipell",
    y_title="Finos (%)",
    y_as_pct=True)
fig1.show()
s3.save_plotly_html(fig1, "finos.html")

In [43]:

fig2 = plot_line(
    df=df_finos_punto, 
    x_col="punto", y_col="malla_6_pct", group_col="producto",
    line_colors=cols_config, order_x=puntos_order, y_as_pct=True,
    show_legend=True 
)
fig2.show()
s3.save_plotly_html(fig2, "malla_6.html")

In [44]:

fig3 = plot_line(
    df=df_finos_punto, 
    x_col="punto", y_col="malla_10_pct",  group_col="producto",
    line_colors=cols_config, order_x=puntos_order, y_as_pct=True,
    show_legend=False 
)

fig3.show()
# 3. Guardar usando tu gestor de S3
s3.save_plotly_html(fig3, "malla_10.html")

In [45]:

fig4 = plot_line(
    df=df_finos_punto, 
    x_col="punto", y_col="malla_18_pct", group_col="producto",
    line_colors=cols_config, order_x=puntos_order, y_as_pct=True,
    show_legend=False 
)
s3.save_plotly_html(fig4, "malla_18.html")

In [46]:
fig_final = create_subplot_grid(
    figures=[fig1, fig3, fig4],
    rows=1, cols=3,
    titles=["Malla 6", "Malla 10", "Malla 12"],
    main_title="Análisis de Finos por Malla",
    shared_y=False, # Compartir eje Y para comparar escalas
    width=1000, height=300
)
fig_final.show()